## Connect to the GHZ database

. . .

```.p
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from scikit-learn.linear_model import LinearRegression

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()
```


## Get data

. . .

```{.p }
df = pd.read_sql(
    """
    select date, ticker, acc, agr, beta, bm, ep, gma, idiovol,
    lev, mom12m, mom1m, mve, operprof, roeq, ret
    from data
    order by date, ticker
    """,
    conn
  )
df = df.dropna()
```

## Split into train and test

. . .

```.p
later = df.index.get_level_values("date")>="2010-01"

train = df[~later]
test = df[later]
```

## Train linear model

. . .

```{.p code-line-numbers="1-4|6-7|9|10"}
predictors = [
  "acc", "agr", "beta", "bm", "ep", "gma", "idiovol",
  "lev", "mom12m", "mom1m", "mve", "operprof", "roeq"
]

Xtrain = train[predictors]
ytrain = train["ret"]

model = LinearRegression()
_ = model.fit(Xtrain, ytrain)
```

## Regression coefficients

. . .

```.p
print(model.intercept_)

coefs = pd.Series(model.coef_, index=predictors)
coefs
```


. . .


In [ ]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" 
database = "ghz"
string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

df = pd.read_sql(
    """
    select date, ticker, acc, agr, beta, bm, ep, gma, idiovol,
    lev, mom12m, mom1m, mve, operprof, roeq, ret
    from data
    order by date, ticker
    """,
    conn
  )
df = df.dropna()
later = df.index.get_level_values("date")>="2010-01"

train = df[~later]
test = df[later]

predictors = [
  "acc", "agr", "beta", "bm", "ep", "gma", "idiovol",
  "lev", "mom12m", "mom1m", "mve", "operprof", "roeq"
]

Xtrain = train[predictors]
ytrain = train["ret"]

model = LinearRegression()
_ = model.fit(Xtrain, ytrain)

print(model.intercept_)

coefs = pd.Series(model.coef_, index=predictors)
coefs